In [60]:
import pandas as pd
from sqlalchemy import create_engine
from config import password

In [175]:
#Dependencies to scrap data
from bs4 import BeautifulSoup
from splinter import Browser
import requests
import numpy as np

### Extract CSVs into DataFrames

In [212]:
car_file_path = "../top_selling_car.csv"
car_data = pd.read_csv(car_file_path)
car_brand = car_data[["#","Car brand","Car model","Release year"]]
car_brand = car_brand.rename(columns ={"#":"top","Car brand":"brand","Car model":"model","Release year":"year_release"})
car_brand["year_2012"]="2012"
car_brand.head()

,top,brand,model,year_release,year_2012
0,1,Toyota,Corolla,1966,2012
1,2,Ford,F-Series,1977,2012
2,3,Volkswagen,Golf,1974,2012
3,4,Volkswagen,Beetle,1938,2012
4,5,Vaz,2101,1970,2012


### Used data from the car_brand table to look for a number of cars (we do 5 in this case) on each  model from Edmunds.com.

In [156]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
#Url to get the good deal of cars
brand=car_brand["brand"].astype(str)
model=car_brand["model"].astype(str)
year=car_brand["year_2012"].astype(str)
new_data  = list() 
many_car = 5
for i in range(0,len(brand)):
    new_data.append(list())
    try:
        url="https://www.edmunds.com/inventory/srp.html?inventorytype=used%2Ccpo&make="+brand[i]+"&model="+model[i]+"&year="+year[i]+"-"+year[i]
        browser.visit(url)
        html = browser.html
        soup = BeautifulSoup(html,'html.parser')
        prices = soup.find_all('h3', class_='mb-0 text-gray-darker')
        results = soup.find_all('li', class_="search-results-inventory-card no-gutters list-unstyled bg-white mb-0_75 mb-lg-1 srp-vehicle-card hover row")
        new_data[i].append(brand[i])
        new_data[i].append(model[i])
        for j in range(0,many_cars):
            href=results[j].a["href"]
            vin=href.split('vin/')[1].split('/')[0]
            price = prices[j].text
            desc=results[j].figure.img["alt"]
            mileage=results[j].find('span', class_="text-gray-darker ml-0_25").text
            if (desc and vin and mileage and price):
                new_data[i].append(desc)
                new_data[i].append(vin)
                new_data[i].append(mileage)
                new_data[i].append(price)
                        
    except (Exception, KeyError) as e:
        print(e)
# Close the browser after scraping
browser.quit()          

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range


### Create a new DataFrame from the web scraping data

In [197]:
new_df=pd.DataFrame(new_data)

### Cleanse DataFrame by deleting rows with no found cars at all.

In [198]:
new_df=pd.DataFrame(new_data)
new_df = df.replace(to_replace='None', value=np.nan).dropna(thresh=3)
new_df.head()

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,Toyota,Corolla,2012 Toyota Corolla LE in Silver,5YFBU4EE4CP018715,"113,119 miles","$6,995",2012 Toyota Corolla LE in Black,5YFBU4EE4CP019427,"90,330 miles","$8,000",...,"130,452 miles","$8,995",2012 Toyota Corolla S in Black,5YFBU4EE0CP029405,"138,476 miles",Not Listed,None,None,None,None
2,Volkswagen,Golf,2012 Volkswagen Golf 2.0L TDI in Black,WVWDM7AJ0CW132117,"58,723 miles","$9,391",2012 Volkswagen Golf 2.0L TDI in White,WVWDM7AJ7CW328023,"73,475 miles","$10,791",...,"72,302 miles","$10,191",2012 Volkswagen Golf 2.0L TDI in Silver,WVWDM7AJXCW334821,"58,916 miles","$12,491",2012 Volkswagen Golf 2.0L TDI in Light Blue,WVWDM7AJ5CW337397,"63,183 miles","$13,900"
3,Volkswagen,Beetle,2012 Volkswagen Beetle 2.5L in Silver,3VWHX7AT1CM636408,"62,220 miles","$6,991",2012 Volkswagen Beetle 2.5L in Dark Brown,3VWJX7AT8CM665776,"84,043 miles","$7,500",...,"74,871 miles","$9,291",2012 Volkswagen Beetle 2.0T Turbo in White,3VW467AT6CM642478,"46,302 miles","$8,400",2012 Volkswagen Beetle 2.0T Turbo PZEV in Gray,3VWVA7AT8CM616631,"94,001 miles","$10,700"
5,Honda,Civic,2012 Honda Civic Si in Black,2HGFG4A51CH701073,"90,066 miles","$9,991",2012 Honda Civic LX in Light Blue,2HGFG3B58CH561239,"6,055 miles","$9,995",...,"88,372 miles","$6,495",2012 Honda Civic Si in Red,2HGFG4A59CH705355,"73,558 miles","$12,750",2012 Honda Civic LX in Silver,2HGFB2F54CH543503,"123,392 miles","$8,413"
7,Honda,Accord,2012 Honda Accord EX in Silver,1HGCS1B72CA015736,"77,372 miles","$8,895",2012 Honda Accord SE in Black,1HGCP2F63CA056546,"107,028 miles","$9,331",...,"48,552 miles","$12,386",2012 Honda Accord EX V-6 in Silver,1HGCP3F73CA004946,"67,727 miles","$11,995",2012 Honda Accord EX in Red,1HGCS1B77CA011469,"80,564 miles","$10,844"


### Create columns's names for new database

In [201]:
new_df.columns = ["brand","model","info_1","vin_1","mil_1","price_1","info_2","vin_2","mil_2","price_2","info_3","vin_3","mil_3","price_3","info_4","vin_4","mil_4","price_4","info_5","vin_5","mil_5","price_5"]
new_df.head()

,brand,model,info_1,vin_1,mil_1,price_1,info_2,vin_2,mil_2,price_2,...,mil_3,price_3,info_4,vin_4,mil_4,price_4,info_5,vin_5,mil_5,price_5
0,Toyota,Corolla,2012 Toyota Corolla LE in Silver,5YFBU4EE4CP018715,"113,119 miles","$6,995",2012 Toyota Corolla LE in Black,5YFBU4EE4CP019427,"90,330 miles","$8,000",...,"130,452 miles","$8,995",2012 Toyota Corolla S in Black,5YFBU4EE0CP029405,"138,476 miles",Not Listed,None,None,None,None
2,Volkswagen,Golf,2012 Volkswagen Golf 2.0L TDI in Black,WVWDM7AJ0CW132117,"58,723 miles","$9,391",2012 Volkswagen Golf 2.0L TDI in White,WVWDM7AJ7CW328023,"73,475 miles","$10,791",...,"72,302 miles","$10,191",2012 Volkswagen Golf 2.0L TDI in Silver,WVWDM7AJXCW334821,"58,916 miles","$12,491",2012 Volkswagen Golf 2.0L TDI in Light Blue,WVWDM7AJ5CW337397,"63,183 miles","$13,900"
3,Volkswagen,Beetle,2012 Volkswagen Beetle 2.5L in Silver,3VWHX7AT1CM636408,"62,220 miles","$6,991",2012 Volkswagen Beetle 2.5L in Dark Brown,3VWJX7AT8CM665776,"84,043 miles","$7,500",...,"74,871 miles","$9,291",2012 Volkswagen Beetle 2.0T Turbo in White,3VW467AT6CM642478,"46,302 miles","$8,400",2012 Volkswagen Beetle 2.0T Turbo PZEV in Gray,3VWVA7AT8CM616631,"94,001 miles","$10,700"
5,Honda,Civic,2012 Honda Civic Si in Black,2HGFG4A51CH701073,"90,066 miles","$9,991",2012 Honda Civic LX in Light Blue,2HGFG3B58CH561239,"6,055 miles","$9,995",...,"88,372 miles","$6,495",2012 Honda Civic Si in Red,2HGFG4A59CH705355,"73,558 miles","$12,750",2012 Honda Civic LX in Silver,2HGFB2F54CH543503,"123,392 miles","$8,413"
7,Honda,Accord,2012 Honda Accord EX in Silver,1HGCS1B72CA015736,"77,372 miles","$8,895",2012 Honda Accord SE in Black,1HGCP2F63CA056546,"107,028 miles","$9,331",...,"48,552 miles","$12,386",2012 Honda Accord EX V-6 in Silver,1HGCP3F73CA004946,"67,727 miles","$11,995",2012 Honda Accord EX in Red,1HGCS1B77CA011469,"80,564 miles","$10,844"


### Reset index 

In [203]:
new_df.reset_index(drop =True, inplace = True)
new_df.head()

,brand,model,info_1,vin_1,mil_1,price_1,info_2,vin_2,mil_2,price_2,...,mil_3,price_3,info_4,vin_4,mil_4,price_4,info_5,vin_5,mil_5,price_5
0,Toyota,Corolla,2012 Toyota Corolla LE in Silver,5YFBU4EE4CP018715,"113,119 miles","$6,995",2012 Toyota Corolla LE in Black,5YFBU4EE4CP019427,"90,330 miles","$8,000",...,"130,452 miles","$8,995",2012 Toyota Corolla S in Black,5YFBU4EE0CP029405,"138,476 miles",Not Listed,None,None,None,None
1,Volkswagen,Golf,2012 Volkswagen Golf 2.0L TDI in Black,WVWDM7AJ0CW132117,"58,723 miles","$9,391",2012 Volkswagen Golf 2.0L TDI in White,WVWDM7AJ7CW328023,"73,475 miles","$10,791",...,"72,302 miles","$10,191",2012 Volkswagen Golf 2.0L TDI in Silver,WVWDM7AJXCW334821,"58,916 miles","$12,491",2012 Volkswagen Golf 2.0L TDI in Light Blue,WVWDM7AJ5CW337397,"63,183 miles","$13,900"
2,Volkswagen,Beetle,2012 Volkswagen Beetle 2.5L in Silver,3VWHX7AT1CM636408,"62,220 miles","$6,991",2012 Volkswagen Beetle 2.5L in Dark Brown,3VWJX7AT8CM665776,"84,043 miles","$7,500",...,"74,871 miles","$9,291",2012 Volkswagen Beetle 2.0T Turbo in White,3VW467AT6CM642478,"46,302 miles","$8,400",2012 Volkswagen Beetle 2.0T Turbo PZEV in Gray,3VWVA7AT8CM616631,"94,001 miles","$10,700"
3,Honda,Civic,2012 Honda Civic Si in Black,2HGFG4A51CH701073,"90,066 miles","$9,991",2012 Honda Civic LX in Light Blue,2HGFG3B58CH561239,"6,055 miles","$9,995",...,"88,372 miles","$6,495",2012 Honda Civic Si in Red,2HGFG4A59CH705355,"73,558 miles","$12,750",2012 Honda Civic LX in Silver,2HGFB2F54CH543503,"123,392 miles","$8,413"
4,Honda,Accord,2012 Honda Accord EX in Silver,1HGCS1B72CA015736,"77,372 miles","$8,895",2012 Honda Accord SE in Black,1HGCP2F63CA056546,"107,028 miles","$9,331",...,"48,552 miles","$12,386",2012 Honda Accord EX V-6 in Silver,1HGCP3F73CA004946,"67,727 miles","$11,995",2012 Honda Accord EX in Red,1HGCS1B77CA011469,"80,564 miles","$10,844"


### Connect to local database

In [189]:
connection_string = "postgres:"+ password+ "@localhost:5432/cars_db"
engine = create_engine(f'postgresql://{connection_string}')

### Check for tables

In [218]:
engine.table_names()

['car_brand', 'car_web']

### Use pandas to load csv converted DataFrame into database

In [213]:
car_brand1=car_brand.drop("year_2012",axis=1)
car_brand1.head()

,top,brand,model,year_release
0,1,Toyota,Corolla,1966
1,2,Ford,F-Series,1977
2,3,Volkswagen,Golf,1974
3,4,Volkswagen,Beetle,1938
4,5,Vaz,2101,1970


### Use pandas to load car_brand DataFrame into database

In [215]:
car_brand1.to_sql(name='car_brand', con=engine, if_exists='append', index=False)

### Use pandas to load new_df DataFrame into database

In [220]:
new_df.to_sql(name='car_web', con=engine, if_exists='append', index=False)

### Query the data from car_brand table in PostgresSQL into pandas

In [221]:
pd.read_sql_query('select * from car_brand', con=engine).head()

,top,brand,model,year_release
0,1,Toyota,Corolla,1966
1,2,Ford,F-Series,1977
2,3,Volkswagen,Golf,1974
3,4,Volkswagen,Beetle,1938
4,5,Vaz,2101,1970


### Query the data from two tables in PostgresSQL into pandas

In [227]:
pd.read_sql_query('select b.top, b.year_release, w.*  from car_brand as b, car_web as w where w.brand=b.brand', con=engine).head()

,top,year_release,brand,model,info_1,vin_1,mil_1,price_1,info_2,vin_2,...,mil_3,price_3,info_4,vin_4,mil_4,price_4,info_5,vin_5,mil_5,price_5
0,1,1966,Toyota,Camry,2012 Toyota Camry XLE,4T1BK1FK8CU517668,"68,331 miles","$11,491",2012 Toyota Camry XLE in Silver,4T4BF1FK4CR212212,...,"108,402 miles","$9,988",2012 Toyota Camry SE in Silver,4T1BF1FK3CU061602,"112,901 miles","$10,988",2012 Toyota Camry SE in Black,4T1BF1FK3CU155706,"87,627 miles","$13,795"
1,1,1966,Toyota,Corolla,2012 Toyota Corolla LE in Silver,5YFBU4EE4CP018715,"113,119 miles","$6,995",2012 Toyota Corolla LE in Black,5YFBU4EE4CP019427,...,"130,452 miles","$8,995",2012 Toyota Corolla S in Black,5YFBU4EE0CP029405,"138,476 miles",Not Listed,None,None,None,None
2,2,1977,Ford,Mustang,2012 Ford Mustang Boss 302 in Yellow,1ZVBP8CU2C5223971,"47,699 miles","$26,791",2012 Ford Mustang GT in Black,1ZVBP8CF5C5263818,...,"83,507 miles","$15,895",2012 Ford Mustang Boss 302 in Orange,1ZVBP8CU3C5243355,"11,617 miles","$33,930",2012 Ford Mustang GT Premium in White,1ZVBP8CF2C5273593,"64,032 miles",Not Listed
3,2,1977,Ford,Focus,2012 Ford Focus SE in Gray,1FAHP3F27CL201346,"88,778 miles","$6,382",2012 Ford Focus SE,1FAHP3F2XCL356828,...,"120,756 miles","$9,599",2012 Ford Focus SEL in Gray,1FAHP3H26CL477272,"83,528 miles","$10,998",2012 Ford Focus SE in Silver,1FAHP3F28CL473212,"81,027 miles","$7,788"
4,2,1977,Ford,Fiesta,2012 Ford Fiesta SE in Black,3FADP4BJ7CM218681,"69,261 miles","$5,990",2012 Ford Fiesta SE in White,3FADP4EJ9CM212182,...,"69,660 miles","$8,000",None,None,None,None,None,None,None,None
